# Chapter MCP: Real Client-Server Split with FastMCP + ADK

Move from **mock** to **real** MCP. Split the Pike Place Fish Market system into a lightweight **client** (ADK agents in this notebook) and a heavy **server** (FastMCP subprocess with vendor logic, memory, learning, and A2A negotiation).

```
┌─────────────────────────────┐      stdio (JSON-RPC)      ┌──────────────────────────────┐
│     CLIENT (This Notebook)  │ ◄──────────────────────────► │   SERVER (pike_place_server)  │
│                             │                              │                              │
│  BuyerAgent (LlmAgent)      │   query_inventory ──────►   │  VENDORS data (5 vendors)    │
│  NegotiatorAgent (LlmAgent) │   check_freshness ──────►   │  PurchaseMemory              │
│  RecorderAgent (LlmAgent)   │   negotiate_price ──────►   │  LearningEngine              │
│                             │   estimate_shipping ─────►   │  NegotiationProtocol (A2A)   │
│  MCPOrchestrator (Python)   │   get_purchase_history ──►   │  ExceptionHandler            │
│  Human Approval Gate        │   run_learning_inference ►   │                              │
│  Display / Reporting        │   record_purchase ───────►   │  7 @mcp.tool() endpoints     │
└─────────────────────────────┘                              └──────────────────────────────┘
```

### Why Split Client and Server?

| Concern | Client (Notebook) | Server (FastMCP) |
|---------|------------------|------------------|
| **Role** | Lightweight orchestration | Heavy computation + state |
| **State** | Stateless (per-session) | Stateful (memory, learning) |
| **Agents** | 3 LlmAgents (Gemini calls) | None — pure Python logic |
| **Scaling** | One notebook per user | One server serves many clients |
| **Updates** | Redeploy client only | Update server tools independently |
| **Protocol** | ADK McpToolset | MCP standard (JSON-RPC over stdio) |

### Patterns on Each Side

| Pattern | Client | Server |
|---------|--------|--------|
| Routing | ✓ Orchestrator routes by fish type | |
| Tool Use | ✓ Agents call MCP tools | ✓ Tools defined with @mcp.tool() |
| ReAct | ✓ BuyerAgent reasons iteratively | |
| Human-in-the-Loop | ✓ Approval gate | |
| Multi-Agent | ✓ 3 specialized agents | |
| A2A Protocol | | ✓ NegotiationProtocol (offer/counter/accept) |
| Memory Management | | ✓ PurchaseMemory (server-side state) |
| Learning & Adaptation | | ✓ LearningEngine (best vendor, avg price) |
| Goal Tracking | ✓ GoalTracker checks | |
| Exception Handling | ✓ Client retries on failure | ✓ Server logs exceptions |
| MCP Server | | ✓ Real FastMCP with 7 tools |

In [1]:
# ── Cell 1: Setup & Dependencies ─────────────────────────
# fastmcp is in requirements.txt — install via: uv pip install -r requirements.txt

import os, json, asyncio, time
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY in .env first"

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.genai import types as genai_types

# Verify fastmcp is installed
import fastmcp
print(f"FastMCP version: {fastmcp.__version__}")

# ADK-native retry (Ch 8b lesson)
RETRY_CONFIG = genai_types.GenerateContentConfig(
    http_options=genai_types.HttpOptions(
        retry_options=genai_types.HttpRetryOptions(
            initial_delay=2,
            attempts=3,
        ),
    ),
)

MODEL = "gemini-2.0-flash"

# ── Display Configuration ────────────────────────────────
# Controls how much output is printed in each phase.
# Set to 0 or None for unlimited output.
DISPLAY_CONFIG = {
    "tool_response_preview": 120,   # chars shown for MCP tool JSON responses
    "agent_summary": 0,             # chars for final agent summary (0 = full)
    "orchestrator_phase": 0,        # chars per phase in MCPOrchestrator (0 = full)
}

def truncate(text: str, limit: int) -> str:
    """Truncate text to limit chars. 0 or None means no truncation."""
    if not limit or len(text) <= limit:
        return text
    return text[:limit] + "..."

print(f"Model: {MODEL}")
print(f"Google API Key: {'set' if os.environ.get('GOOGLE_API_KEY') else 'MISSING'}")
print(f"Display config: {DISPLAY_CONFIG}")

FastMCP version: 3.0.2
Model: gemini-2.0-flash
Google API Key: set
Display config: {'tool_response_preview': 120, 'agent_summary': 0, 'orchestrator_phase': 0}


In [6]:
%%writefile pike_place_server.py
#!/usr/bin/env python3
"""
Pike Place Fish Market — MCP Server

A real FastMCP server that exposes vendor operations as MCP tools.
Runs as a subprocess. ADK clients connect via stdio (JSON-RPC 2.0).

Server-side patterns:
  - A2A Negotiation Protocol (structured offer/counter/accept)
  - ADK SqliteSessionService for persistent purchase memory
  - Learning Engine (recommends best vendors from persisted data)
  - Exception Handling (logs failures, suggests recovery)

Persistence:
  Purchase history and vendor scores are stored in a SQLite database
  via ADK's SqliteSessionService. This means data survives across
  subprocess restarts — each new stdio connection picks up where the
  last one left off.

Tools exposed:
  1. query_inventory     — check vendor stock (stateless)
  2. check_freshness     — quality assessment (stateless)
  3. negotiate_price     — A2A negotiation (in-memory per process)
  4. estimate_shipping   — cost + delivery time (stateless)
  5. get_purchase_history — query persisted memory (ADK state)
  6. run_learning_inference — best vendor recommendation (ADK state)
  7. record_purchase     — save to persisted memory (ADK state)
"""

import json, os
from typing import Optional, List, Dict
from datetime import datetime
from fastmcp import FastMCP

# ADK persistence imports
from google.adk.sessions.sqlite_session_service import SqliteSessionService
from google.adk.events.event import Event
from google.adk.events.event_actions import EventActions

# ═══════════════════════════════════════════════════════════
# VENDOR DATA (same 5 Pike Place vendors from Capstone)
# ═══════════════════════════════════════════════════════════

VENDORS = {
    "Wild Salmon Co": {
        "specialty": ["King Salmon", "Sockeye Salmon", "Coho Salmon"],
        "inventory": {"King Salmon": 25, "Sockeye Salmon": 40, "Coho Salmon": 15},
        "price_per_lb": {"King Salmon": 28, "Sockeye Salmon": 22, "Coho Salmon": 19},
        "freshness_score": {"King Salmon": 9, "Sockeye Salmon": 10, "Coho Salmon": 8},
        "reliability": 0.95,
        "min_margin": 0.20,
        "cost_basis": {"King Salmon": 18, "Sockeye Salmon": 14, "Coho Salmon": 12},
    },
    "Atlantic Fishery": {
        "specialty": ["King Salmon", "Atlantic Salmon"],
        "inventory": {"King Salmon": 50, "Atlantic Salmon": 100},
        "price_per_lb": {"King Salmon": 18, "Atlantic Salmon": 14},
        "freshness_score": {"King Salmon": 6, "Atlantic Salmon": 7},
        "reliability": 0.90,
        "min_margin": 0.15,
        "cost_basis": {"King Salmon": 12, "Atlantic Salmon": 9},
    },
    "Halibut House": {
        "specialty": ["Pacific Halibut", "Lingcod"],
        "inventory": {"Pacific Halibut": 12, "Lingcod": 20},
        "price_per_lb": {"Pacific Halibut": 35, "Lingcod": 24},
        "freshness_score": {"Pacific Halibut": 9, "Lingcod": 8},
        "reliability": 0.85,
        "min_margin": 0.25,
        "cost_basis": {"Pacific Halibut": 22, "Lingcod": 15},
    },
    "Tuna King": {
        "specialty": ["Yellowfin Tuna", "Albacore Tuna"],
        "inventory": {"Yellowfin Tuna": 30, "Albacore Tuna": 45},
        "price_per_lb": {"Yellowfin Tuna": 22, "Albacore Tuna": 16},
        "freshness_score": {"Yellowfin Tuna": 8, "Albacore Tuna": 7},
        "reliability": 0.92,
        "min_margin": 0.18,
        "cost_basis": {"Yellowfin Tuna": 14, "Albacore Tuna": 10},
    },
    "Crab Cart": {
        "specialty": ["Dungeness Crab", "King Crab Legs"],
        "inventory": {"Dungeness Crab": 35, "King Crab Legs": 8},
        "price_per_lb": {"Dungeness Crab": 16, "King Crab Legs": 45},
        "freshness_score": {"Dungeness Crab": 9, "King Crab Legs": 7},
        "reliability": 0.88,
        "min_margin": 0.22,
        "cost_basis": {"Dungeness Crab": 10, "King Crab Legs": 30},
    },
}


# ═══════════════════════════════════════════════════════════
# ADK PERSISTED MEMORY (SqliteSessionService)
# ═══════════════════════════════════════════════════════════
#
# Instead of in-memory PurchaseMemory/LearningEngine classes,
# we use ADK's SqliteSessionService for disk-based persistence.
# Each subprocess reads/writes the same SQLite file, so data
# survives across subprocess restarts.

DB_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)),
                       "pike_place_memory.db")
APP_NAME = "pike_place_market"
USER_ID = "market_buyer"
SESSION_ID = "purchase_memory"

session_service = SqliteSessionService(db_path=DB_PATH)


async def _get_or_create_session():
    """Get existing memory session or create one with empty state."""
    session = await session_service.get_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    if not session:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID,
            state={"purchase_history": [], "vendor_scores": {}})
    return session


async def _update_state(session, **state_delta):
    """Write state changes to SQLite via ADK Event + append_event."""
    event = Event(
        author="pike_place_server",
        invocation_id=f"inv_{datetime.now().isoformat()}",
        actions=EventActions(state_delta=state_delta))
    await session_service.append_event(session, event)


def _compute_memory_summary(history, vendor_scores):
    """Compute summary stats from persisted data."""
    vendors = {}
    for vendor, scores in vendor_scores.items():
        vendors[vendor] = {
            "purchases": len(scores),
            "avg_reliability": round(sum(scores) / len(scores), 3),
        }
    return {"total_purchases": len(history), "vendors": vendors}


def _compute_learning_insights(fish_type, history):
    """Compute learning recommendations from persisted history."""
    fish_history = [h for h in history if h["fish_type"] == fish_type]
    if not fish_history:
        return {
            "fish_type": fish_type,
            "status": "NO_HISTORY",
            "message": f"No past purchases for {fish_type}.",
        }
    # Best vendor by freshness-to-price ratio
    vendor_scores = {}
    for h in fish_history:
        v = h["vendor"]
        score = h["freshness"] / max(h["price_per_lb"], 1)
        vendor_scores.setdefault(v, []).append(score)
    avg_scores = {v: sum(s) / len(s) for v, s in vendor_scores.items()}
    best = max(avg_scores, key=avg_scores.get)
    avg_price = round(
        sum(h["price_per_lb"] for h in fish_history) / len(fish_history), 2)
    return {
        "fish_type": fish_type,
        "status": "HAS_HISTORY",
        "past_purchases": len(fish_history),
        "best_vendor": best,
        "average_price_per_lb": avg_price,
        "best_freshness_seen": max(h["freshness"] for h in fish_history),
        "persistence": "ADK SqliteSessionService",
    }


# ═══════════════════════════════════════════════════════════
# A2A NEGOTIATION PROTOCOL (in-memory per subprocess)
# ═══════════════════════════════════════════════════════════
#
# Negotiation state is intentionally NOT persisted — each
# negotiation is a fresh conversation within one subprocess.

class NegotiationProtocol:
    """Structured A2A protocol for buyer-vendor negotiation.

    Tracks rounds and the last counter price so that re-offering
    at or above the counter results in ACCEPTED (not another counter).
    """

    MAX_ROUNDS = 5  # server-enforced cap

    def __init__(self):
        self.rounds: List[Dict] = []
        self.last_counter: Optional[float] = None

    def reset(self):
        self.rounds = []
        self.last_counter = None

    @property
    def round_count(self) -> int:
        """Number of OFFER rounds (buyer submissions) so far."""
        return sum(1 for r in self.rounds if r["type"] == "OFFER")

    def add_round(self, msg_type: str, sender: str, **kwargs) -> Dict:
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": msg_type,
            "from": sender,
            "round": len(self.rounds) + 1,
            **kwargs,
        }
        self.rounds.append(msg)
        if msg_type == "COUNTER_OFFER":
            self.last_counter = kwargs.get("counter_price")
        return msg

    def trace(self) -> List[Dict]:
        return self.rounds.copy()


protocol = NegotiationProtocol()


# ═══════════════════════════════════════════════════════════
# FASTMCP SERVER + TOOLS
# ═══════════════════════════════════════════════════════════

mcp = FastMCP("Pike Place Vendor Server")


# ── Batch tool (reduces 10+ round-trips to 1) ────────────

@mcp.tool()
def get_market_snapshot(fish_type: str, qty_lbs: float = 1.0) -> str:
    """Get a complete market snapshot for a fish type in ONE call.

    Returns inventory, freshness, quality grade, and shipping estimate
    for ALL vendors that carry this fish type. Use this instead of
    calling query_inventory + check_freshness + estimate_shipping
    individually for each vendor — it's 10x faster.

    Args:
        fish_type: Fish type to search for (e.g. 'King Salmon')
        qty_lbs: Quantity in pounds (for shipping estimate)
    """
    results = []
    for vendor_name, vendor in VENDORS.items():
        if fish_type not in vendor["inventory"]:
            continue
        freshness = vendor["freshness_score"][fish_type]
        quality = ("EXCELLENT" if freshness >= 9 else "GOOD" if freshness >= 7
                   else "FAIR" if freshness >= 5 else "POOR")
        shipping_cost = round(5 + qty_lbs * 1.0, 2)
        delivery_days = 1 if vendor_name in ["Wild Salmon Co", "Atlantic Fishery"] else 2
        results.append({
            "vendor": vendor_name,
            "fish_type": fish_type,
            "available_lbs": vendor["inventory"][fish_type],
            "price_per_lb": vendor["price_per_lb"][fish_type],
            "freshness_score": freshness,
            "quality_grade": quality,
            "sashimi_suitable": freshness >= 8,
            "reliability": vendor["reliability"],
            "shipping_cost": shipping_cost,
            "delivery_days": delivery_days,
        })
    if not results:
        return json.dumps({
            "fish_type": fish_type,
            "status": "NOT_FOUND",
            "message": f"No vendors carry {fish_type}",
            "all_vendors": list(VENDORS.keys()),
        })
    # Sort by freshness desc, then price asc
    results.sort(key=lambda r: (-r["freshness_score"], r["price_per_lb"]))
    return json.dumps({
        "fish_type": fish_type,
        "qty_lbs": qty_lbs,
        "vendors_found": len(results),
        "snapshot": results,
    })


# ── Stateless tools (read VENDORS dict only) ─────────────

@mcp.tool()
def query_inventory(vendor_name: str, fish_type: str) -> str:
    """Query a specific vendor's inventory for a fish type.

    Returns JSON with availability, price, freshness, and reliability.
    Use this to check what a vendor has in stock before buying.

    Args:
        vendor_name: Vendor name (e.g. 'Wild Salmon Co', 'Atlantic Fishery',
                     'Halibut House', 'Tuna King', 'Crab Cart')
        fish_type: Fish type (e.g. 'King Salmon', 'Pacific Halibut')
    """
    if vendor_name not in VENDORS:
        return json.dumps({"error": f"Unknown vendor: {vendor_name}",
                           "available_vendors": list(VENDORS.keys())})

    vendor = VENDORS[vendor_name]
    if fish_type not in vendor["inventory"]:
        return json.dumps({"vendor": vendor_name, "fish_type": fish_type,
                           "status": "NOT_CARRIED",
                           "carried_fish": vendor["specialty"]})

    return json.dumps({
        "vendor": vendor_name,
        "fish_type": fish_type,
        "available_lbs": vendor["inventory"][fish_type],
        "price_per_lb": vendor["price_per_lb"][fish_type],
        "freshness_score": vendor["freshness_score"][fish_type],
        "reliability": vendor["reliability"],
        "status": "AVAILABLE",
    })


@mcp.tool()
def check_freshness(vendor_name: str, fish_type: str) -> str:
    """Get detailed freshness assessment for a vendor's fish.

    Returns quality grade (EXCELLENT/GOOD/FAIR/POOR) and whether
    the fish is suitable for sashimi (requires freshness >= 8).

    Args:
        vendor_name: Vendor name
        fish_type: Fish type
    """
    inv = json.loads(query_inventory(vendor_name, fish_type))
    if "error" in inv or inv.get("status") != "AVAILABLE":
        return json.dumps({"error": f"{vendor_name} does not have {fish_type}",
                           "detail": inv})

    freshness = inv["freshness_score"]
    quality = ("EXCELLENT" if freshness >= 9 else "GOOD" if freshness >= 7
               else "FAIR" if freshness >= 5 else "POOR")

    return json.dumps({
        "vendor": vendor_name,
        "fish_type": fish_type,
        "freshness_score": freshness,
        "quality_grade": quality,
        "sashimi_suitable": freshness >= 8,
        "recommendation": ("Perfect for sashimi" if freshness >= 9
                           else "Acceptable for sashimi" if freshness >= 8
                           else "Cook only — not fresh enough for raw"),
    })


@mcp.tool()
def estimate_shipping(vendor_name: str, qty_lbs: float) -> str:
    """Estimate shipping cost and delivery time from a Pike Place vendor.

    All vendors use overnight cold pack shipping.
    Premium vendors (Wild Salmon Co, Atlantic Fishery) deliver in 1 day.
    Others deliver in 2 days.

    Args:
        vendor_name: Vendor name
        qty_lbs: Quantity in pounds
    """
    cost = round(5 + qty_lbs * 1.0, 2)
    days = 1 if vendor_name in ["Wild Salmon Co", "Atlantic Fishery"] else 2
    return json.dumps({
        "vendor": vendor_name,
        "shipping_cost": cost,
        "delivery_days": days,
        "method": "overnight cold pack",
    })


# ── A2A Negotiation (in-memory per subprocess) ───────────

@mcp.tool()
def negotiate_price(vendor_name: str, fish_type: str,
                    qty_lbs: float, offer_price_per_lb: float) -> str:
    """Run A2A negotiation protocol with a vendor.

    The server tracks negotiation state across calls. Each call is one round:
    1. Buyer sends OFFER at offer_price_per_lb
    2. Vendor evaluates and responds:
       - ACCEPTED: offer >= list price, or offer >= last counter, or max rounds hit
       - COUNTER_OFFER: offer above minimum but below acceptance threshold
       - REJECTED: offer below vendor's minimum price

    Max 5 rounds enforced server-side. To accept a counter-offer,
    call again with offer_price_per_lb >= the counter price.

    Args:
        vendor_name: Vendor to negotiate with
        fish_type: Fish type to buy
        qty_lbs: Quantity in pounds
        offer_price_per_lb: Your proposed price per pound
    """
    if vendor_name not in VENDORS:
        return json.dumps({"error": f"Unknown vendor: {vendor_name}"})

    vendor = VENDORS[vendor_name]
    if fish_type not in vendor["cost_basis"]:
        return json.dumps({"error": f"{vendor_name} does not carry {fish_type}"})

    cost = vendor["cost_basis"][fish_type]
    min_price = round(cost * (1 + vendor["min_margin"]), 2)
    list_price = vendor["price_per_lb"][fish_type]

    # Reset if: no prior rounds, different vendor/fish, or prior negotiation concluded
    concluded = any(r["type"] in ("ACCEPTED", "REJECTED") for r in protocol.rounds)
    is_new = (not protocol.rounds
              or concluded
              or protocol.rounds[0].get("vendor") != vendor_name
              or protocol.rounds[0].get("fish_type") != fish_type)
    if is_new:
        protocol.reset()

    # Record the buyer's offer
    protocol.add_round("OFFER", "BuyerAgent",
                       vendor=vendor_name, fish_type=fish_type,
                       qty_lbs=qty_lbs, offer_price=offer_price_per_lb)

    # ── Accept conditions ────────────────────────────
    # 1. Offer at or above list price → accept at list price
    # 2. Offer at or above the last counter → accept at counter price
    # 3. Max rounds reached and offer >= min → accept at offer price
    if offer_price_per_lb >= list_price:
        final_price = list_price
        protocol.add_round("ACCEPTED", vendor_name,
                           final_price=final_price, qty=qty_lbs)
        return json.dumps({
            "status": "ACCEPTED",
            "final_price_per_lb": final_price,
            "qty_lbs": qty_lbs,
            "total_cost": round(final_price * qty_lbs, 2),
            "a2a_trace": protocol.trace(),
        })

    if (protocol.last_counter is not None
            and offer_price_per_lb >= protocol.last_counter):
        final_price = protocol.last_counter
        protocol.add_round("ACCEPTED", vendor_name,
                           final_price=final_price, qty=qty_lbs,
                           reason="Offer meets previous counter")
        return json.dumps({
            "status": "ACCEPTED",
            "final_price_per_lb": final_price,
            "qty_lbs": qty_lbs,
            "total_cost": round(final_price * qty_lbs, 2),
            "a2a_trace": protocol.trace(),
        })

    if (protocol.round_count >= protocol.MAX_ROUNDS
            and offer_price_per_lb >= min_price):
        final_price = offer_price_per_lb
        protocol.add_round("ACCEPTED", vendor_name,
                           final_price=final_price, qty=qty_lbs,
                           reason=f"Max rounds ({protocol.MAX_ROUNDS}) reached")
        return json.dumps({
            "status": "ACCEPTED",
            "final_price_per_lb": final_price,
            "qty_lbs": qty_lbs,
            "total_cost": round(final_price * qty_lbs, 2),
            "a2a_trace": protocol.trace(),
        })

    # ── Counter or reject ────────────────────────────
    if offer_price_per_lb >= min_price:
        counter = round((offer_price_per_lb + list_price) / 2, 2)
        protocol.add_round("COUNTER_OFFER", vendor_name,
                           counter_price=counter, available_qty=qty_lbs)
        return json.dumps({
            "status": "COUNTER_OFFER",
            "counter_price_per_lb": counter,
            "available_qty": qty_lbs,
            "vendor_min_price": min_price,
            "rounds_used": protocol.round_count,
            "max_rounds": protocol.MAX_ROUNDS,
            "a2a_trace": protocol.trace(),
        })

    else:
        protocol.add_round("REJECTED", vendor_name,
                           reason=f"Offer ${offer_price_per_lb}/lb below minimum ${min_price}/lb")
        return json.dumps({
            "status": "REJECTED",
            "reason": f"Offer ${offer_price_per_lb}/lb is below vendor minimum ${min_price}/lb",
            "vendor_min_price": min_price,
            "a2a_trace": protocol.trace(),
        })


# ── Persisted tools (ADK SqliteSessionService) ───────────

@mcp.tool()
async def get_purchase_history(fish_type: str) -> str:
    """Retrieve purchase history from ADK-persisted server memory.

    Returns past purchases for the specified fish type,
    including vendor, price, freshness, and timestamps.
    Data persists across server restarts via SqliteSessionService.

    Args:
        fish_type: Fish type to look up (e.g. 'King Salmon')
    """
    session = await _get_or_create_session()
    history = session.state.get("purchase_history", [])
    vendor_scores = session.state.get("vendor_scores", {})

    fish_history = [h for h in history if h["fish_type"] == fish_type]
    return json.dumps({
        "fish_type": fish_type,
        "count": len(fish_history),
        "records": fish_history[-5:],  # last 5
        "memory_summary": _compute_memory_summary(history, vendor_scores),
        "persistence": "ADK SqliteSessionService",
    })


@mcp.tool()
async def run_learning_inference(fish_type: str) -> str:
    """Query the server-side learning engine for buying recommendations.

    Analyzes past purchases (persisted in ADK SqliteSessionService)
    to recommend the best vendor based on quality-to-price ratio.
    Also returns average price paid and best freshness score seen.

    Args:
        fish_type: Fish type to get recommendations for
    """
    session = await _get_or_create_session()
    history = session.state.get("purchase_history", [])
    return json.dumps(_compute_learning_insights(fish_type, history))


@mcp.tool()
async def record_purchase(vendor_name: str, fish_type: str, qty_lbs: float,
                          price_per_lb: float, freshness_score: int) -> str:
    """Record a completed purchase to ADK-persisted server memory.

    This updates vendor reliability scores and enables the
    learning engine to make better recommendations next time.
    Data persists via SqliteSessionService — survives server restarts.

    Args:
        vendor_name: Vendor purchased from
        fish_type: Fish type bought
        qty_lbs: Quantity in pounds
        price_per_lb: Final negotiated price per pound
        freshness_score: Freshness rating 1-10
    """
    session = await _get_or_create_session()
    history = session.state.get("purchase_history", [])
    vendor_scores = session.state.get("vendor_scores", {})

    # Build new record
    record = {
        "vendor": vendor_name, "fish_type": fish_type, "qty": qty_lbs,
        "price_per_lb": price_per_lb,
        "total_cost": round(qty_lbs * price_per_lb, 2),
        "freshness": freshness_score, "on_time": True,
        "timestamp": datetime.now().isoformat(),
    }
    history.append(record)

    # Update vendor reliability score
    score = (freshness_score / 10) * 1.0  # on_time=True
    vendor_scores.setdefault(vendor_name, []).append(score)

    # Persist to SQLite via ADK event
    await _update_state(session,
                        purchase_history=history,
                        vendor_scores=vendor_scores)

    return json.dumps({
        "status": "RECORDED",
        "record": record,
        "memory_state": _compute_memory_summary(history, vendor_scores),
        "persistence": "ADK SqliteSessionService",
    })


# ═══════════════════════════════════════════════════════════
# RUN SERVER
# ═══════════════════════════════════════════════════════════

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--sse", action="store_true",
                        help="Run as persistent SSE server (for orchestrator)")
    parser.add_argument("--port", type=int, default=8765,
                        help="Port for SSE server (default: 8765)")
    parser.add_argument("--host", default="localhost")
    args = parser.parse_args()

    if args.sse:
        mcp.run(transport="sse", host=args.host, port=args.port,
                show_banner=False)
    else:
        mcp.run()  # stdio (default for individual cells)


Overwriting pike_place_server.py


---
## What the Server Exposes

The `pike_place_server.py` file we just wrote is a **real MCP server** using FastMCP. When ADK starts it as a subprocess, it communicates over **stdio using JSON-RPC 2.0** — the standard MCP protocol.

| Tool | Purpose | Server-Side Pattern |
|------|---------|--------------------|
| `query_inventory` | Check vendor stock | Vendor data lookup |
| `check_freshness` | Quality assessment | Calls query_inventory internally |
| `negotiate_price` | Price negotiation | **A2A Protocol** (OFFER → COUNTER/ACCEPT/REJECT) |
| `estimate_shipping` | Delivery cost/time | Simple calculation |
| `get_purchase_history` | Past purchases | **PurchaseMemory** (server state) |
| `run_learning_inference` | Vendor recommendation | **LearningEngine** (analyzes history) |
| `record_purchase` | Save purchase | **PurchaseMemory** update |

The client (this notebook) never sees vendor data, memory, or negotiation logic. It only sees tool names and JSON responses.

In [7]:
# ── Cell 4: Create ADK Client Agents with McpToolset ──────
#
# These agents are LIGHTWEIGHT — no vendor data, no memory,
# no negotiation logic. They just call MCP tools on the server.

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# Connection config: ADK will start pike_place_server.py as a subprocess
# and communicate over stdio (JSON-RPC 2.0)
MCP_CONNECTION = StdioConnectionParams(
    server_params=StdioServerParameters(
        command="python",
        args=["pike_place_server.py"],
    ),
    timeout=10.0,
)

# ── Buyer Agent ──────────────────────────────────────
# Discovers options by querying the MCP server
buyer_agent = LlmAgent(
    name="BuyerAgent",
    model=MODEL,
    instruction="""
You are a fish buyer at Pike Place Market. Given a customer request:

1. Call get_market_snapshot with the fish type to get ALL vendor data
   (inventory, freshness, shipping) in a single call.
2. If a snapshot isn't available, fall back to individual tool calls:
   query_inventory for each vendor, then check_freshness, then estimate_shipping.

Return a RANKED list of options with:
- Vendor name, price/lb, freshness score, quality grade, shipping cost
- Your recommendation for which vendor is best and why.
""",
    tools=[
        McpToolset(connection_params=MCP_CONNECTION),
    ],
    generate_content_config=RETRY_CONFIG,
)

# ── Learner Agent ────────────────────────────────────
# Focused agent: queries learning engine ONLY — no discovery
learner_agent = LlmAgent(
    name="LearnerAgent",
    model=MODEL,
    instruction="""
You query the learning engine for purchase history insights.

RULES:
1. Call run_learning_inference with the fish type provided.
2. Report EXACTLY what the learning engine returns — nothing more.
3. Do NOT call any other tools (no query_inventory, no check_freshness).
4. If the result says NO_HISTORY, say so clearly.
5. If the result says HAS_HISTORY, report: best vendor, average price,
   past purchases count, and best freshness seen.
""",
    tools=[
        McpToolset(connection_params=MCP_CONNECTION),
    ],
    generate_content_config=RETRY_CONFIG,
)

# ── Negotiator Agent ─────────────────────────────────
# Handles price negotiation via MCP server's A2A protocol
negotiator_agent = LlmAgent(
    name="NegotiatorAgent",
    model=MODEL,
    instruction="""
You are a tough but fair price negotiator for Pike Place Market.

STRATEGY — always try to get the best deal:
1. Start with your initial offer (the price given to you).
2. If you get a COUNTER_OFFER, DO NOT immediately accept it.
   Instead, offer a price BETWEEN your last offer and the counter
   (split the difference). This shows you're willing to negotiate.
3. If you get another COUNTER_OFFER, you may accept it by offering
   at or above the counter price — but only after at least 2 rounds
   of back-and-forth.
4. The server enforces a max of 5 rounds, so don't worry about
   going forever.

RULES:
- Never offer above your target price.
- Always call negotiate_price to make each offer — that's how
  the server tracks the A2A protocol.
- To ACCEPT a counter, call negotiate_price with offer >= the counter.
- Report the final deal with: price, total cost, rounds, and A2A trace.
""",
    tools=[
        McpToolset(connection_params=MCP_CONNECTION),
    ],
    generate_content_config=RETRY_CONFIG,
)

# ── Recorder Agent ───────────────────────────────────
# Records purchases to server-side memory
recorder_agent = LlmAgent(
    name="RecorderAgent",
    model=MODEL,
    instruction="""
You record completed purchases to the server's memory.

Given purchase details (vendor, fish type, quantity, price, freshness):
1. Call record_purchase with the exact details provided.
2. Confirm the recording was successful.
3. Report the updated memory state.
""",
    tools=[
        McpToolset(connection_params=MCP_CONNECTION),
    ],
    generate_content_config=RETRY_CONFIG,
)

print("Client agents created:")
print(f"  BuyerAgent      — discovers vendors via MCP (uses get_market_snapshot)")
print(f"  LearnerAgent    — queries learning engine ONLY (1 MCP call)")
print(f"  NegotiatorAgent — negotiates prices via A2A on server (up to 5 rounds)")
print(f"  RecorderAgent   — records purchases to server memory")
print(f"\nMCP connection: StdioConnectionParams → pike_place_server.py (timeout: 10s)")
print(f"Model: {MODEL} with auto-retry on 429s")

Client agents created:
  BuyerAgent      — discovers vendors via MCP (uses get_market_snapshot)
  LearnerAgent    — queries learning engine ONLY (1 MCP call)
  NegotiatorAgent — negotiates prices via A2A on server (up to 5 rounds)
  RecorderAgent   — records purchases to server memory

MCP connection: StdioConnectionParams → pike_place_server.py (timeout: 10s)
Model: gemini-2.0-flash with auto-retry on 429s


---
## Phase 1: Discovery via MCP

**Patterns: Tool Use + ReAct + MCP Protocol**

The BuyerAgent calls `query_inventory`, `check_freshness`, and `estimate_shipping` — all served by the FastMCP server running as a subprocess. The agent reasons iteratively (ReAct) over the tool responses.

```
BuyerAgent (notebook)                    MCP Server (subprocess)
    │                                         │
    │── query_inventory(Wild Salmon Co) ──────►│── looks up VENDORS dict
    │◄── {status: AVAILABLE, price: 28} ──────│
    │                                         │
    │── check_freshness(Wild Salmon Co) ──────►│── computes quality grade
    │◄── {freshness: 9, sashimi: true} ───────│
    │                                         │
    │── estimate_shipping(Wild Salmon Co) ────►│── calculates cost
    │◄── {cost: 10, days: 1} ─────────────────│
```

Watch the tool calls in the output — each one goes over the MCP stdio protocol to the server.

In [8]:
# ── Phase 1: Run Discovery via MCP Server ────────────────

async def run_discovery(customer_request: str) -> str:
    """BuyerAgent discovers vendor options via real MCP calls."""
    print(f"{'='*60}")
    print(f"CUSTOMER: {customer_request}")
    print(f"{'='*60}\n")

    session_service = InMemorySessionService()
    runner = Runner(
        agent=buyer_agent,
        app_name="mcp_discovery",
        session_service=session_service,
    )
    session = await session_service.create_session(
        app_name="mcp_discovery", user_id="buyer")

    content = types.Content(
        role="user",
        parts=[types.Part.from_text(text=customer_request)])

    response_text = ""
    tool_calls = 0

    async for event in runner.run_async(
        user_id="buyer", session_id=session.id, new_message=content
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "function_call") and part.function_call:
                    tool_calls += 1
                    args = dict(part.function_call.args) if part.function_call.args else {}
                    print(f"  [MCP Call {tool_calls}] {part.function_call.name}({args})")
                elif hasattr(part, "function_response") and part.function_response:
                    resp = truncate(str(part.function_response.response),
                                    DISPLAY_CONFIG["tool_response_preview"])
                    print(f"      → {resp}")
                elif part.text and part.text.strip():
                    response_text += part.text
        if event.is_final_response():
            break

    print(f"\n{'─'*60}")
    print(f"BUYER SUMMARY ({tool_calls} MCP tool calls):")
    print(f"{'─'*60}")
    print(truncate(response_text, DISPLAY_CONFIG["agent_summary"]))
    return response_text


# Run discovery
discovery_result = await run_discovery(
    "I need 5 lbs of fresh King Salmon for a dinner party this Saturday. "
    "Budget is $150. It needs to be sashimi-grade."
)

CUSTOMER: I need 5 lbs of fresh King Salmon for a dinner party this Saturday. Budget is $150. It needs to be sashimi-grade.



/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_toolset.py:313: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/genai/_api_client.py:765: DeprecationWarning: Inheritance class AiohttpClientSession from ClientSession is discouraged
  class AiohttpClientSession(aiohttp.ClientSession):  # type: ignore[misc]


  [MCP Call 1] get_market_snapshot({'qty_lbs': 5, 'fish_type': 'King Salmon'})
      → {'content': [{'type': 'text', 'text': '{"fish_type": "King Salmon", "qty_lbs": 5.0, "vendors_found": 2, "snapshot": [{"v...

────────────────────────────────────────────────────────────
BUYER SUMMARY (1 MCP tool calls):
────────────────────────────────────────────────────────────
Okay, I can help you find the best King Salmon. First, I'll get a snapshot of the market to see who has it in stock.
Here's what I found:

*   **Wild Salmon Co:** Price is $28/lb, freshness score is 9 (EXCELLENT, sashimi suitable), and shipping is $10 (1 day).
*   **Atlantic Fishery:** Price is $18/lb, freshness score is 6 (FAIR, not sashimi suitable), and shipping is $10 (1 day).

Given your budget of $150 and need for sashimi-grade salmon, Wild Salmon Co is the only viable option. 5 lbs will cost $140 plus $10 shipping, fitting within your budget. Atlantic Fishery is cheaper, but the quality is too low for sashimi.

**Reco

---
## Phase 2: Negotiation via A2A Protocol on Server

**Patterns: A2A Protocol + MCP + Tool Use**

The NegotiatorAgent calls `negotiate_price` — but the actual negotiation logic runs **on the server**. The server's `NegotiationProtocol` generates a structured OFFER → COUNTER_OFFER → ACCEPTED/REJECTED trace.

The client never sees vendor cost basis or margin calculations. It only sees the protocol result.

**Server-side acceptance rules:**
1. Offer at or above **list price** → instant ACCEPT at list price
2. Offer at or above **last counter** → ACCEPT at counter price (this is how the agent "agrees" to a counter)
3. **Max 5 rounds** reached and offer above minimum → ACCEPT at offer price (safety cap)

**Agent strategy:** The NegotiatorAgent is instructed to haggle — it splits the difference between its last offer and the counter rather than immediately accepting. This produces a realistic multi-round negotiation.

```
NegotiatorAgent (client)              MCP Server
    │                                     │
    │── negotiate_price($23.80/lb) ──────►│
    │                                     │── Round 1: OFFER $23.80
    │                                     │── Vendor: $23.80 > min $21.60 ✓
    │◄── {COUNTER_OFFER: $25.90/lb} ─────│── Counter = midpoint($23.80, $28)
    │                                     │
    │── negotiate_price($24.85/lb) ──────►│  (agent splits difference)
    │                                     │── Round 2: OFFER $24.85
    │◄── {COUNTER_OFFER: $26.43/lb} ─────│── Counter = midpoint($24.85, $28)
    │                                     │
    │── negotiate_price($26.43/lb) ──────►│  (agent accepts counter)
    │                                     │── Round 3: OFFER $26.43
    │                                     │── $26.43 >= last counter $26.43 ✓
    │◄── {ACCEPTED: $26.43/lb} ──────────│
```

In [9]:
# ── Phase 2: Negotiate Price via MCP ──────────────────────

async def run_negotiation(vendor_name: str, fish_type: str,
                          qty: float, target_price: float) -> str:
    """NegotiatorAgent runs price negotiation via MCP A2A."""
    print(f"\n{'='*60}")
    print(f"NEGOTIATION: {qty}lb {fish_type} from {vendor_name}")
    print(f"Target: ${target_price}/lb")
    print(f"{'='*60}\n")

    session_service = InMemorySessionService()
    runner = Runner(
        agent=negotiator_agent,
        app_name="mcp_negotiate",
        session_service=session_service,
    )
    session = await session_service.create_session(
        app_name="mcp_negotiate", user_id="negotiator")

    content = types.Content(
        role="user",
        parts=[types.Part.from_text(
            text=f"Negotiate with {vendor_name} for {qty}lb {fish_type}. "
                 f"Target price: ${target_price}/lb. "
                 f"Start your offer at ${target_price * 0.85:.2f}/lb (15% below target). "
                 f"Try up to 3 rounds. Report the final deal and A2A trace."
        )])

    response_text = ""
    tool_calls = 0

    async for event in runner.run_async(
        user_id="negotiator", session_id=session.id, new_message=content
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "function_call") and part.function_call:
                    tool_calls += 1
                    args = dict(part.function_call.args) if part.function_call.args else {}
                    print(f"  [A2A Call {tool_calls}] {part.function_call.name}({args})")
                elif hasattr(part, "function_response") and part.function_response:
                    resp = truncate(str(part.function_response.response),
                                    DISPLAY_CONFIG["tool_response_preview"])
                    print(f"      → {resp}")
                elif part.text and part.text.strip():
                    response_text += part.text
        if event.is_final_response():
            break

    print(f"\n{'─'*60}")
    print(f"NEGOTIATION RESULT ({tool_calls} rounds):")
    print(f"{'─'*60}")
    print(truncate(response_text, DISPLAY_CONFIG["agent_summary"]))
    return response_text


# Negotiate with Wild Salmon Co
negotiation_result = await run_negotiation(
    vendor_name="Wild Salmon Co",
    fish_type="King Salmon",
    qty=5.0,
    target_price=28.0,
)


NEGOTIATION: 5.0lb King Salmon from Wild Salmon Co
Target: $28.0/lb



/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_toolset.py:313: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/genai/_api_client.py:765: DeprecationWarning: Inheritance class AiohttpClientSession from ClientSession is discouraged
  class AiohttpClientSession(aiohttp.ClientSession):  # type: ignore[misc]


  [A2A Call 1] negotiate_price({'fish_type': 'King Salmon', 'offer_price_per_lb': 23.8, 'qty_lbs': 5, 'vendor_name': 'Wild Salmon Co'})
      → {'content': [{'type': 'text', 'text': '{"status": "COUNTER_OFFER", "counter_price_per_lb": 25.9, "available_qty": 5.0, "...
  [A2A Call 2] negotiate_price({'offer_price_per_lb': 24.85, 'vendor_name': 'Wild Salmon Co', 'fish_type': 'King Salmon', 'qty_lbs': 5})
      → {'content': [{'type': 'text', 'text': '{"status": "COUNTER_OFFER", "counter_price_per_lb": 26.43, "available_qty": 5.0, ...
  [A2A Call 3] negotiate_price({'fish_type': 'King Salmon', 'offer_price_per_lb': 26.43, 'vendor_name': 'Wild Salmon Co', 'qty_lbs': 5})
      → {'content': [{'type': 'text', 'text': '{"status": "ACCEPTED", "final_price_per_lb": 26.43, "qty_lbs": 5.0, "total_cost":...

────────────────────────────────────────────────────────────
NEGOTIATION RESULT (3 rounds):
────────────────────────────────────────────────────────────
Okay, I will negotiate with Wild Salmon 

---
## Phase 3: Record Purchase & Query Learning

**Patterns: Memory Management + Learning & Adaptation (both on server)**

After a successful purchase, the RecorderAgent calls `record_purchase` to save it to server-side memory. Then we query `run_learning_inference` to see what the server has learned.

```
RecorderAgent (client)               MCP Server
    │                                     │
    │── record_purchase(Wild Salmon...) ──►│── PurchaseMemory.add_purchase()
    │◄── {status: RECORDED} ──────────────│── vendor_scores updated
    │                                     │
    │── run_learning_inference(King..) ───►│── LearningEngine.get_insights()
    │◄── {best_vendor: Wild Salmon Co} ──│── analyzed from memory
```

In [10]:
# ── Phase 3: Record Purchase + Query Learning ─────────────

async def run_record_and_learn(
    vendor: str, fish_type: str, qty: float,
    price: float, freshness: int
) -> str:
    """Record a purchase and then query learning insights."""
    print(f"\n{'='*60}")
    print(f"RECORD & LEARN: {qty}lb {fish_type} from {vendor}")
    print(f"{'='*60}\n")

    session_service = InMemorySessionService()

    # Step 1: Record the purchase
    print("[RECORDER] Saving purchase to server memory...")
    recorder_runner = Runner(
        agent=recorder_agent, app_name="mcp_record",
        session_service=session_service)
    session = await session_service.create_session(
        app_name="mcp_record", user_id="recorder")

    msg = types.Content(role="user", parts=[types.Part.from_text(
        text=f"Record this purchase: vendor={vendor}, fish_type={fish_type}, "
             f"qty_lbs={qty}, price_per_lb={price}, freshness_score={freshness}")])

    record_response = ""
    async for event in recorder_runner.run_async(
        user_id="recorder", session_id=session.id, new_message=msg):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "function_call") and part.function_call:
                    print(f"  [MCP] {part.function_call.name}(...)")
                elif part.text and part.text.strip():
                    record_response += part.text
        if event.is_final_response():
            break

    print(f"  ✓ {truncate(record_response, DISPLAY_CONFIG['agent_summary'])}")

    # Step 2: Query learning insights
    print(f"\n[BUYER] Querying learning insights...")
    buyer_runner = Runner(
        agent=buyer_agent, app_name="mcp_learn",
        session_service=session_service)
    learn_session = await session_service.create_session(
        app_name="mcp_learn", user_id="learner")

    learn_msg = types.Content(role="user", parts=[types.Part.from_text(
        text=f"What has the system learned about {fish_type}? "
             f"Use run_learning_inference to check.")])

    learn_response = ""
    async for event in buyer_runner.run_async(
        user_id="learner", session_id=learn_session.id, new_message=learn_msg):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "function_call") and part.function_call:
                    print(f"  [MCP] {part.function_call.name}(...)")
                elif part.text and part.text.strip():
                    learn_response += part.text
        if event.is_final_response():
            break

    print(f"\n{'─'*60}")
    print(f"LEARNING INSIGHTS:")
    print(f"{'─'*60}")
    print(truncate(learn_response, DISPLAY_CONFIG["agent_summary"]))
    return learn_response


# Record a purchase and query what the server learned
learn_result = await run_record_and_learn(
    vendor="Wild Salmon Co",
    fish_type="King Salmon",
    qty=5.0,
    price=28.0,
    freshness=9,
)


RECORD & LEARN: 5.0lb King Salmon from Wild Salmon Co

[RECORDER] Saving purchase to server memory...


/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_toolset.py:313: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/genai/_api_client.py:765: DeprecationWarning: Inheritance class AiohttpClientSession from ClientSession is discouraged
  class AiohttpClientSession(aiohttp.ClientSession):  # type: ignore[misc]


  [MCP] record_purchase(...)
  ✓ The purchase was recorded successfully. Here is the updated memory state: total_purchases=4, vendors={'Wild Salmon Co': {'purchases': 3, 'avg_reliability': 0.9}, 'Crab Cart': {'purchases': 1, 'avg_reliability': 0.9}}

[BUYER] Querying learning insights...
  [MCP] run_learning_inference(...)

────────────────────────────────────────────────────────────
LEARNING INSIGHTS:
────────────────────────────────────────────────────────────
Okay, I have the learning engine's recommendation for King Salmon. It says that there is purchase history for King Salmon, with 3 past purchases recorded. The best vendor based on that history is Wild Salmon Co, with an average price paid of $27.08 per pound and the best freshness score seen being 9.



---
## Full Pipeline: MCPOrchestrator with ADK Persisted Memory

### Design Principle: Client vs Server Agents

| | **Server Agent (Heavy)** | **Client Agent (Light)** |
|---|---|---|
| **State** | Owns persistence (`SqliteSessionService`) | Stateless — no memory |
| **Logic** | Business rules, A2A protocol, learning | Orchestration, display, approval |
| **Data** | Vendor inventory, purchase history in SQLite | Only what MCP tools return |
| **Tools** | `@mcp.tool()` with ADK state read/write | `McpToolset` — just calls server |
| **Lifecycle** | Subprocess may restart — state survives via SQLite | Ephemeral per Runner call |
| **When to use** | Complex domain logic, data persistence, multi-step protocols | Thin orchestration, user-facing interaction |

### Best Practices

1. **Server owns truth** — client never stores purchase data
2. **Server tools are self-contained** — each tool reads/writes its own state via ADK
3. **Client trusts server responses** — doesn't duplicate validation
4. **Negotiation stays in-memory** — per-connection, resets naturally (no persistence needed)
5. **Memory/learning use ADK state** → survives subprocess restarts via SQLite

### How persistence works

```
Subprocess 1 (Phase 1-3)          Subprocess 2 (Phase 5)
  record_purchase() ──writes──►  ┌──────────────────┐  ◄──reads── run_learning_inference()
                                 │ pike_place_memory  │
                                 │      .db           │
                                 │ (SqliteSession     │
                                 │  Service)          │
                                 └──────────────────┘
                                 Survives restarts!
```

Each stdio subprocess connects to the **same SQLite file**. ADK's `SqliteSessionService`
handles all the read/write/event machinery. No SSE, no background processes needed.

In [11]:
# ── Full Pipeline: MCPOrchestrator ────────────────────────
# Uses simple stdio connections. Each _call_agent() may start a
# new server subprocess, but purchase memory PERSISTS via ADK's
# SqliteSessionService (SQLite file on disk).

import os, re

MEMORY_DB = "pike_place_memory.db"

# Set to True  → fresh start every run (good for clean demos)
# Set to False → accumulate history across runs (watch learning improve!)
FRESH_START = True

if FRESH_START and os.path.exists(MEMORY_DB):
    os.remove(MEMORY_DB)
    print(f"  ♻ Reset {MEMORY_DB} (FRESH_START=True)")
elif os.path.exists(MEMORY_DB):
    import sqlite3, json as _json
    conn = sqlite3.connect(MEMORY_DB)
    row = conn.execute(
        "SELECT state FROM sessions WHERE id = ?", ("purchase_memory",)
    ).fetchone()
    conn.close()
    if row:
        n = len(_json.loads(row[0]).get("purchase_history", []))
        print(f"  📚 Keeping {MEMORY_DB} — {n} past purchases (FRESH_START=False)")
    else:
        print(f"  📚 Keeping {MEMORY_DB} (FRESH_START=False)")


def _parse_negotiated_price(negotiation_text: str, fallback: float) -> float:
    """Extract the final negotiated price from the agent's response.

    Handles Gemini escaped underscores (final_price variants) and various
    prose formats (Price: $26.82/lb, Price per lb: $26.43, etc.)
    """
    # Normalize: strip backslash-escapes that Gemini adds
    clean = negotiation_text.replace("\\_", "_")
    # Try A2A trace: 'final_price': 26.82
    match = re.search(r'["\'"]final_price["\'"]:\s*([\d.]+)', clean)
    if match:
        return float(match.group(1))
    # Try prose: "Price: $26.82/lb" or "Price per lb: $26.43"
    match = re.search(r"[Pp]rice(?:\s+per\s+lb)?[=:]\s*\$?([\d.]+)", clean)
    if match:
        return float(match.group(1))
    return fallback


class MCPOrchestrator:
    """Orchestrates the Pike Place buying pipeline over MCP.

    Client-side only — lightweight. All complex logic and
    persistence runs on the server (SqliteSessionService).

    Fixes applied:
    - Phase 2 uses LearnerAgent (1 call) not BuyerAgent (10 calls)
    - Phase 5 records actual negotiated price, not target price
    - Vendor selection influenced by learning data
    """

    async def _call_agent(self, agent: LlmAgent, message: str,
                          label: str = "AGENT") -> str:
        """Send a message to an agent and collect the response."""
        session_service = InMemorySessionService()
        runner = Runner(
            agent=agent, app_name="mcp_pipeline",
            session_service=session_service)
        session = await session_service.create_session(
            app_name="mcp_pipeline", user_id="orchestrator")

        content = types.Content(
            role="user",
            parts=[types.Part.from_text(text=message)])

        response = ""
        tool_calls = 0
        async for event in runner.run_async(
            user_id="orchestrator", session_id=session.id,
            new_message=content
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, "function_call") and part.function_call:
                        tool_calls += 1
                        name = part.function_call.name
                        print(f"  [{label}] MCP call {tool_calls}: {name}")
                    elif part.text and part.text.strip():
                        response += part.text
            if event.is_final_response():
                break

        return response.strip()

    async def run(self, fish_type: str, qty: float,
                  budget: float, target_price: float,
                  vendor: str = "Wild Salmon Co",
                  auto_approve: bool = True) -> dict:
        """Execute the full buying pipeline."""

        phase_limit = DISPLAY_CONFIG["orchestrator_phase"]

        print(f"\n{'#'*60}")
        print(f"# MCP PIPELINE")
        print(f"# {qty}lb {fish_type}, budget ${budget}")
        print(f"{'#'*60}")

        # ── Phase 1: Discovery ──────────────────────────
        print(f"\n┌─ Phase 1: DISCOVERY (BuyerAgent → MCP)")
        discovery = await self._call_agent(
            buyer_agent,
            f"Find all vendors with {fish_type}. Check freshness and shipping. "
            f"Budget is ${budget} for {qty}lb. Which vendor is best?",
            label="BUYER")
        print(f"│  Discovery complete")
        print(f"│  {truncate(discovery, phase_limit)}")

        # ── Phase 2: Learning (FIX: uses LearnerAgent) ──
        print(f"│")
        print(f"├─ Phase 2: LEARNING (LearnerAgent → 1 MCP call)")
        insights = await self._call_agent(
            learner_agent,
            f"Check what we know about {fish_type}.",
            label="LEARN")
        print(f"│  {truncate(insights, phase_limit)}")

        # Act on learning — prove the system uses prior experience
        if "best_vendor" in insights.lower() or "HAS_HISTORY" in insights:
            print(f"│  🎯 Learning active — system is using prior purchases to guide decisions")
            # FIX 4: Let learning influence vendor selection
            # Extract recommended vendor from learning insights
            best_match = re.search(r"[Bb]est vendor[: ]+([ A-Za-z]+)", insights)
            if best_match:
                learned_vendor = best_match.group(1).strip()
                if learned_vendor != vendor:
                    print(f"│  🔄 Learning recommends {learned_vendor} "
                          f"(overriding default: {vendor})")
                    vendor = learned_vendor
                else:
                    print(f"│  ✓ Learning confirms {vendor} is the best choice")
        else:
            print(f"│  📝 No prior data yet — this will be the baseline purchase")

        # ── Phase 3: Negotiation ────────────────────────
        print(f"│")
        print(f"├─ Phase 3: NEGOTIATION (A2A on server)")
        offer_price = round(target_price * 0.85, 2)
        negotiation = await self._call_agent(
            negotiator_agent,
            f"Negotiate with {vendor} for {qty}lb {fish_type}. "
            f"Start at ${offer_price}/lb. Target: ${target_price}/lb. "
            f"Try up to 3 rounds.",
            label="NEGOTIATE")
        print(f"│  {truncate(negotiation, phase_limit)}")

        # FIX 2: Parse actual negotiated price from agent response
        actual_price = _parse_negotiated_price(negotiation, target_price)
        if actual_price != target_price:
            print(f"│  💰 Negotiated ${actual_price}/lb "
                  f"(saved ${round(target_price - actual_price, 2)}/lb)")

        # ── Human Approval Gate ─────────────────────────
        print(f"│")
        print(f"├─ Phase 4: HUMAN APPROVAL")
        if auto_approve:
            print(f"│  ✅ Auto-approved (set auto_approve=False for manual)")
            approved = True
        else:
            response = input(f"│  Approve purchase from {vendor}? (yes/no): ")
            approved = response.strip().lower() in ["yes", "y"]

        if not approved:
            print(f"└─ ❌ REJECTED by customer")
            return {"status": "REJECTED"}

        # ── Phase 5: Record Purchase (FIX: uses actual price) ──
        print(f"│")
        print(f"├─ Phase 5: RECORD (persist to ADK SqliteSessionService)")
        record = await self._call_agent(
            recorder_agent,
            f"Record purchase: vendor={vendor}, fish_type={fish_type}, "
            f"qty_lbs={qty}, price_per_lb={actual_price}, freshness_score=9",
            label="RECORD")
        print(f"│  ✓ Purchase persisted to SQLite via ADK")

        # ── Done ────────────────────────────────────────
        deal = {
            "status": "COMPLETE",
            "vendor": vendor,
            "fish_type": fish_type,
            "qty": qty,
            "price_per_lb": actual_price,
            "total": round(qty * actual_price, 2),
        }
        print(f"│")
        print(f"└─ ✅ PIPELINE COMPLETE")
        print(f"   {deal}")
        return deal


print("MCPOrchestrator ready (4 fixes applied)")
print("  ✓ Phase 2: LearnerAgent (1 call, not 10)")
print("  ✓ Phase 5: records actual negotiated price")
print("  ✓ Phase 1: BuyerAgent uses get_market_snapshot")
print("  ✓ Learning influences vendor selection")

  ♻ Reset pike_place_memory.db (FRESH_START=True)
MCPOrchestrator ready (4 fixes applied)
  ✓ Phase 2: LearnerAgent (1 call, not 10)
  ✓ Phase 5: records actual negotiated price
  ✓ Phase 1: BuyerAgent uses get_market_snapshot
  ✓ Learning influences vendor selection


In [12]:
# ── Scenario 1: Happy Path ────────────────────────────────

orchestrator = MCPOrchestrator()

deal1 = await orchestrator.run(
    fish_type="King Salmon",
    qty=5.0,
    budget=150.0,
    target_price=28.0,
    vendor="Wild Salmon Co",
)


############################################################
# MCP PIPELINE
# 5.0lb King Salmon, budget $150.0
############################################################

┌─ Phase 1: DISCOVERY (BuyerAgent → MCP)
  [BUYER] MCP call 1: get_market_snapshot
│  Discovery complete
│  Here's a breakdown of the King Salmon options:

**Wild Salmon Co:**
*   Price: $28/lb
*   Freshness: 9 (EXCELLENT, suitable for sashimi)
*   Shipping: $10
*   Total cost: (5 lbs * $28/lb) + $10 = $150

**Atlantic Fishery:**
*   Price: $18/lb
*   Freshness: 6 (FAIR, not suitable for sashimi)
*   Shipping: $10
*   Total cost: (5 lbs * $18/lb) + $10 = $100

**Recommendation:**

Given your budget of $150, both vendors fit. However, Wild Salmon Co. offers significantly better quality (EXCELLENT vs FAIR) and freshness. The total cost for Wild Salmon Co. is exactly $150. I recommend Wild Salmon Co. for the better quality and freshness, and it fits within your budget.
│
├─ Phase 2: LEARNING (LearnerAgent → 1 MCP call

/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_toolset.py:313: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/Users/jerrykuru/dev/ai/learn/agent/agent_adk/.venv/lib/python3.12/site-packages/google/genai/_api_client.py:765: DeprecationWarning: Inheritance class AiohttpClientSession from ClientSession is discouraged
  class AiohttpClientSession(aiohttp.ClientSession):  # type: ignore[misc]


  [LEARN] MCP call 1: run_learning_inference
│  NO_HISTORY
│  📝 No prior data yet — this will be the baseline purchase
│
├─ Phase 3: NEGOTIATION (A2A on server)
  [NEGOTIATE] MCP call 1: negotiate_price
  [NEGOTIATE] MCP call 2: negotiate_price
  [NEGOTIATE] MCP call 3: negotiate_price
│  Okay, I will start negotiating with Wild Salmon Co for 5.0lb of King Salmon, starting at $23.8/lb. My target is $28.0/lb.
They countered with $25.9/lb. I will offer a price between my last offer and their counter, which is $24.85/lb.
They countered with $26.43/lb. I will offer $26.43 to accept.
I have reached a deal with Wild Salmon Co. The final price is $26.43/lb, the total cost is $132.15, and it took 3 rounds of negotiation. The A2A trace is: [{'protocol': 'pike_place_a2a_v1', 'type': 'OFFER', 'from': 'BuyerAgent', 'round': 1, 'vendor': 'Wild Salmon Co', 'fish_type': 'King Salmon', 'qty_lbs': 5.0, 'offer_price': 23.8}, {'protocol': 'pike_place_a2a_v1', 'type': 'COUNTER_OFFER', 'from': 'Wild Salmon

In [13]:
# ── Scenario 2: Learning Demo (2nd purchase) ──────────────
# The server subprocess restarts, but the SQLite database
# (pike_place_memory.db) persists Scenario 1's purchase.
# run_learning_inference WILL see the King Salmon purchase!

print("\nScenario 2: Second purchase — ADK SqliteSessionService preserves memory")
print("BuyerAgent will call run_learning_inference and see past purchases.\n")

deal2 = await orchestrator.run(
    fish_type="King Salmon",
    qty=3.0,
    budget=100.0,
    target_price=28.0,
    vendor="Wild Salmon Co",
)


Scenario 2: Second purchase — ADK SqliteSessionService preserves memory
BuyerAgent will call run_learning_inference and see past purchases.


############################################################
# MCP PIPELINE
# 3.0lb King Salmon, budget $100.0
############################################################

┌─ Phase 1: DISCOVERY (BuyerAgent → MCP)
  [BUYER] MCP call 1: get_market_snapshot
│  Discovery complete
│  Here's a breakdown of the King Salmon options, considering your budget of $100 for 3 lbs:

**Wild Salmon Co:**

*   Price: $28/lb
*   Freshness: 9 (EXCELLENT, suitable for sashimi)
*   Shipping: $8
*   Total cost: (3 lbs \* $28/lb) + $8 = $92

**Atlantic Fishery:**

*   Price: $18/lb
*   Freshness: 6 (FAIR, not suitable for sashimi)
*   Shipping: $8
*   Total cost: (3 lbs \* $18/lb) + $8 = $62

**Recommendation:**

Both vendors are within your budget. However, **Wild Salmon Co** is the better choice. While slightly more expensive, their King Salmon has an "EXCELLENT" qu

In [14]:
# ── Scenario 3: Different fish, different vendor ──────────

deal3 = await orchestrator.run(
    fish_type="Dungeness Crab",
    qty=4.0,
    budget=80.0,
    target_price=16.0,
    vendor="Crab Cart",
)


############################################################
# MCP PIPELINE
# 4.0lb Dungeness Crab, budget $80.0
############################################################

┌─ Phase 1: DISCOVERY (BuyerAgent → MCP)
  [BUYER] MCP call 1: get_market_snapshot
│  Discovery complete
│  Crab Cart is the only vendor with Dungeness Crab. They have 35lbs available at $16/lb. The freshness score is 9, quality is EXCELLENT, and it's suitable for sashimi. Shipping cost is $9.0 and takes 2 days.

The total cost for 4lbs would be (4 * $16) + $9 = $73, which is within your budget.

Recommendation: Crab Cart is the best option, as they are the only vendor with Dungeness Crab and the total cost is within your budget.
│
├─ Phase 2: LEARNING (LearnerAgent → 1 MCP call)
  [LEARN] MCP call 1: run_learning_inference
│  NO_HISTORY
│  📝 No prior data yet — this will be the baseline purchase
│
├─ Phase 3: NEGOTIATION (A2A on server)
  [NEGOTIATE] MCP call 1: negotiate_price
  [NEGOTIATE] MCP call 2: negotiat

---
## Verify: Inspect the SQLite Database

The cell below queries `pike_place_memory.db` directly to prove
that ADK's `SqliteSessionService` persisted all purchases across
the 3 scenarios — even though each phase used a separate subprocess.

In [15]:
# ── Verify: Inspect ADK's SQLite persistence ─────────────
import sqlite3, json

db_path = "pike_place_memory.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Show tables created by ADK
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = [row[0] for row in cursor.fetchall()]
print(f"ADK tables in {db_path}: {tables}")

# Show the persisted session state
cursor.execute("SELECT state FROM sessions WHERE id = ?", ("purchase_memory",))
row = cursor.fetchone()
if row:
    state = json.loads(row[0])
    history = state.get("purchase_history", [])
    scores = state.get("vendor_scores", {})
    print(f"\nTotal purchases persisted: {len(history)}")
    print(f"Vendors with scores: {list(scores.keys())}")
    print(f"\n── Purchase History ──")
    for i, h in enumerate(history, 1):
        print(f"  {i}. {h['vendor']} | {h['fish_type']} | "
              f"{h['qty']}lb @ ${h['price_per_lb']}/lb | "
              f"freshness={h['freshness']} | {h['timestamp'][:19]}")
else:
    print("No session found — run the scenarios first!")

conn.close()
print(f"\n✓ All data persisted via ADK SqliteSessionService")

ADK tables in pike_place_memory.db: ['app_states', 'user_states', 'sessions', 'events']

Total purchases persisted: 3
Vendors with scores: ['Wild Salmon Co', 'Crab Cart']

── Purchase History ──
  1. Wild Salmon Co | King Salmon | 5.0lb @ $26.43/lb | freshness=9 | 2026-02-27T13:25:09
  2. Wild Salmon Co | King Salmon | 3.0lb @ $26.43/lb | freshness=9 | 2026-02-27T13:25:28
  3. Crab Cart | Dungeness Crab | 4.0lb @ $15.1/lb | freshness=9 | 2026-02-27T13:25:46

✓ All data persisted via ADK SqliteSessionService
